In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

In [36]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
   # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata= pd.read_csv('movies_metadata.csv', low_memory=False)
    ratings= pd.read_csv('ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open('wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Television series' not in movie)]
    len(wiki_movies)

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    movies= [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    movies_df= pd.DataFrame(movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        movies_df['imdb_id'] = movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        movies_df.drop_duplicates(subset= 'imdb_id', inplace= True)
    except Exception as e:
        print(e)

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    jso= [x for x in movies_df.columns if movies_df[x].isnull().sum() < len(movies_df) * 0.9]
    keep_movies_df = movies_df[jso]
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office= box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one= r'\$\d+\.?\d*\s*[mb]illion'
 

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two= r'\$\d{1,3}(?:,\d{3})+'

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
            value = float(s) * 10**6

        # return value
            return value

    # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
            value = float(s) * 10**9

        # return value
            return value

    # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
            s = re.sub('\$|,','', s)

        # convert to float
            value = float(s)

        # return value
            return value

    # otherwise, return NaN
        else:
            return np.nan
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    movies_df['Budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    movies_df.drop('Running time', axis=1, inplace=True)
    # 2. Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    # 3. Merged the two DataFrames into the movies DataFrame.
    merged_df = pd.merge(movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    merged_df.drop(columns=['title_wiki','release_date_wiki','Language'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(merged_df, kaggle_column, wiki_column):
        merged_df[kaggle_column] = merged_df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
        merged_df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    for col in merged_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = merged_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
        if num_values == 1:
            print(col)

    # 7. Filter the movies DataFrame for specific columns.
    merged_df= merged_df.loc[:,'id', 'title_kaggle','url','budget_kaggle',
                        'release_date_kaggle','Country','Distributor','Producer(s)','Director'
                        'Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on']
    # 8. Rename the columns in the movies DataFrame.
    merged_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)
    
  

    
    # 9. Transform and merge the ratings DataFrame.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    return wiki_movies_df, movies_with_ratings_df, movies_df
    

In [37]:
# 10. Create the path to your file directory and variables for the three files.
# The Wikipedia data
wiki_file = f'wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'ratings.csv'

In [38]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

Also known as
Opening theme
Biographical data
Born
Died
Resting place
Occupation
Years active
Spouse(s)
Children
Parent(s)
Genres
Instruments
Labels
Website
Type
Industry
Fate
Founded
Founder
Headquarters
Parent
Released
Recorded
Venue
Length
Label
Director
Producer
Area
Coordinates
Status
Opening date
Closing date
Replaced
Replaced by
Name
Attraction type
Music
Duration
Simplified Chinese
Traditional Chinese
Hanyu Pinyin
Literal meaning
Transcriptions
Bopomofo
Gwoyeu Romatzyh
Wade–Giles
IPA
Yale Romanization
Jyutping
Hokkien POJ
Engine(s)
Genre(s)
Actor control
Production company
Release(s)
Format(s)
Published
Dewey Decimal
Text
Illustrator
Published in English
Developed by
Ending theme
No. of seasons
Nationality
Portrayed by
Alias
Species
Gender
Family
Alma mater
Camera setup
Novel(s)
Comics
Film(s)
Developer(s)
Publisher(s)
Designer(s)
Programmer(s)
Artist(s)
Writer(s)
Engine
Platform(s)
Release
Mode(s)
Full name
Height
Seasons
Other names
Relatives
Yiddish
Formerly
Key people
Total

IndexingError: Too many indexers

In [25]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [26]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

AttributeError: 'str' object has no attribute 'head'

In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [ ]:
# 15. Check the movies_df DataFrame. 
movies_df.head()